In [1]:
import numpy as np
import os
import pandas as pd
import pickle as pk
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.backend import categorical_crossentropy
from tensorflow.keras.preprocessing import sequence
from nltk.translate.bleu_score import sentence_bleu

In [2]:
# Conversational Model Metric
def perplexity(y_true, y_pred):
    cross_entropy = categorical_crossentropy(y_pred, y_true)
    return pow(2, cross_entropy)

In [3]:
os.chdir('../../logs/')

In [5]:
# Especify model directory
modelDir = 'logs_trained_models_server/model_epochs-250_batch-64_hidden-128_embed-128_v4'



In [23]:
# Load variables
word2index_inp = pk.load(open(os.path.join(modelDir,'word2index_inp.pk'), 'rb'))
index2word_inp = pk.load(open(os.path.join(modelDir,'index2word_inp.pk'),'rb'))

word2index_out = pk.load(open(os.path.join(modelDir,'word2index_out.pk'), 'rb'))
index2word_out = pk.load(open(os.path.join(modelDir,'index2word_out.pk'),'rb'))

X_input = pk.load(open(os.path.join(modelDir,'x_inp.pk'),'rb'))
y_input = pk.load(open(os.path.join(modelDir,'y_inp.pk'),'rb'))


In [6]:
# Load models
model = load_model(os.path.join(modelDir, 'chpts/39_2.60.chpt'), custom_objects={'perplexity': perplexity})
encoder_model = load_model(os.path.join(modelDir, 'encoder_model_128_64_250_128.h5'))
decoder_model = load_model(os.path.join(modelDir, 'decoder_model_128_64_250_128.h5'))

OSError: Unable to open file (unable to open file: name = 'logs_trained_models_server/model_epochs-250_batch-64_hidden-128_embed-128_v4/chpts/39_2.60.chpt', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [25]:
def decode_sequence(input_seq, target_len, v=0, counter=None, n=None):
    '''Generates answer from given sequence of word indexes'''
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, target_len))
    # Populate the first token of target sequence with the start token.
    target_seq[0, 0] = word2index_out['GO']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decode_sentence = []
    prediction_index = 0
    while not stop_condition:
        ## Print sequence for debugging
        #print(target_seq)
        output_word_index, h, c = decoder_model.predict([target_seq] + states_value)
        print(output_word_index.shape)

        # Sample a token
        sampled_word_index = np.argmax(output_word_index[0, -1, :])
        sampled_word = index2word_out[sampled_word_index]
        decode_sentence.append(sampled_word)

        # Exit condition: either hit max length
        # or find stop token.
        if (sampled_word == 'EOS' or len(decode_sentence) >= target_len):
            stop_condition = True
        else:
            # Update the target sequence (of length 1).
            target_seq[0, prediction_index+1] = sampled_word_index
            prediction_index += 1

    ## Print sentece for debugging
    # print(' '.join(decode_sentence))

    # Update states
    states_value = [h, c]
    
    if v == 1:
        counter += 1
        print('{} of {}'.format(counter, n), end='\r')

    return ' '.join(decode_sentence)

In [26]:
def indexseq2text(indexes_seq, index2word):
    string_ = []
    for val in indexes_seq:
        string_.append(index2word[val])
    
    return ' '.join(string_)

In [27]:
result_df = pd.DataFrame()
result_df['input_string']  = [indexseq2text(X_input[i], index2word_inp) for i in range(X_input.shape[0])]
result_df['real_string'] = [indexseq2text(y_input[i], index2word_out) for i in range(y_input.shape[0])]

In [28]:
result_df['predictions_string'] = [decode_sequence(X_input[i:i+1], y_input.shape[1], 1, i, y_input.shape[0]) for i in range(X_input.shape[0]-1)]




(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2

KeyboardInterrupt: 

In [29]:
np.random.seed(0)
sample = np.random.randint(0, y_input.shape[0]-1, size=10)
X_sample, y_sample = X_input[sample, :], y_input[sample, :]

In [30]:
with open('test_samples.txt','wb') as f:
    sep = '-'*80
    for i in range(X_input.shape[0]):
        input_string = indexseq2text(X_input[i], index2word_inp)
        print(input_string)
        print(sep)
        real_string = indexseq2text(y_input[i], index2word_out)
        print(real_string)
        if i < X_input.shape[0]-1:
            predicted_string = decode_sequence(X_input[i:i+1], y_input.shape[1])
        print(predicted_string)
        
            
        
        f.write()

result_df = pd.DataFrame()
result_df['input_string']  = [indexseq2text(X_input[i], index2word_inp) for i in range(X_input.shape[0])]
result_df['real_string'] = [indexseq2text(y_input[i], index2word_out) for i in range(y_input.shape[0])]

Buenos dias , Una consulta , ? Donde podre ver cuando , a que hora y en que salon debo rendir mis examenes parciales ? Mis profesores ya me han dado las fechas , pero no estaban UNK de la hora y el salon . Gracias Atte . UNK UNK PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
--------------------------------------------------------------------------------
GO Estimada Ariana Las UNK de los salones se UNK UNK hora antes UNK en el UNK en los UNK .

KeyboardInterrupt: 

In [31]:
result_df['predictions_string'] = [decode_sequence(X_input[i:i+1], y_input.shape[1], 1, i, y_input.shape[0]) for i in range(X_input.shape[0]-1)]




(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)


KeyboardInterrupt: 

In [40]:
sep = '-'*80
for i in range(X_sample.shape[0]):
    input_string = indexseq2text(X_sample[i], index2word_inp)
    print(input_string)
    print(sep)
    real_string = indexseq2text(y_sample[i], index2word_out)
    print(real_string)
    if i < X_input.shape[0]-1:
        predicted_string = decode_sequence(X_sample[i:i+1], y_sample.shape[1])
    print(X_sample[i:i+1])
    print(predicted_string)
    print('*'*80)

Buenas noches , yo me UNK a la carrera de UNK hace un ano ; sin embargo , me aparecen como obligatorios cursos de la carrera de UNK que UNK antes . De derecho solo me aparecen 4 cursos . Espero su respuesta , me encuentro UNK . Muchas gracias de antemano , UNK UNK PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD
--------------------------------------------------------------------------------
GO Correo atendido . Saludos , PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD

[[ 46  39   3 132  25  23 143   1   5 941  48  15   6  63   2 187  12 419
    4  47  38  27   1   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
blackboard estas ceremonia ceremonia mediante mediante mediante encuentras encuentras encuentras Segundo Segundo Segundo Segundo atender atender atender atender atender atender solicitudes solicitudes solicitudes solicitudes vemos vemos vemos Examen Examen Examen Examen electivo electivo 24 24 24 24 24 24 Rocio Rocio Rocio Jorge Jorge Jorge Jorge Jorge Jorge Jorge Jorge Gabriela Gabriela una una una una una una una una Te cuerpo cuerpo cuerpo cuerpo cuerpo Karina Karina Karina Ka

KeyboardInterrupt: 

In [37]:
decode_sequence(np.array([range(200)]), 200)

(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2004)
(1, 200, 2

'reclamos solicitar lamentamos intercambio intercambio intercambio intercambio intercambio junto junto junto junto junto junto junto junto junto Francisco Francisco Francisco Francisco S/.20.00 S/.20.00 S/.20.00 S/.20.00 S/.20.00 S/.20.00 S/.20.00 S/.20.00 S/.20.00 S/.20.00 profesores profesores indico indico indico indico indico indico 16:30 este este este este este este administrativos administrativos administrativos P.D P.D P.D P.D P.D P.D P.D P.D volver volver Atentamente Atentamente Atentamente te te te te te te grado grado grado grado grado grado grado grado grado Guillermo Guillermo Guillermo Guillermo Guillermo Guillermo podido podido podido podido podido podido ser ser ser ser ser ser ser ser ser ser listado listado aplico aplico credito credito credito emitira emitira emitira emitira emitira 21 21 21 21 21 21 21 21 21 durante Mario Mario Mario Mario Mario muestra muestra muestra muestra muestra muestra muestra muestra muestra 1000 1000 Asistente Asistente Asistente confirmaci